

# 2015-11-27


The purpose of the these test was to see whether it is possible to at all see APT transmissions using an RTL-SDR dongle and it's original antenna, without any modifications. 

## Test 1

- pass: NOAA-18 2015-11-27 @ 16:00 UTC
- frequency: monitoring entire {137..138} MHz APT band
- antenna: 10cm whip
- software: reception using *Gqrx*.


## Test 2

- pass: NOAA-15 2015-11-27 @ 16:40 UTC
- frequency: monitoring entire {137..138} MHz APT band
- antenna: 10cm whip
- sofware: gqrx


---

There was no trace of APT transmissions in either test, this was expected.


# 2015-12-2

Setting up equipment for reception and some link budget calculation. Started designing the RF front-end filter for the band.

## Gear

This test was only an attempt to see the downlink transmission on a spectrum analyzer. A Rhode & Schwarz FSH 4 portable spectrum analyzer used as the "receiver". The recepetion attempt was made using a is a log-periogic antenna.

**TODO:** get the spec for the antenna

## Contacts

### Two failed attempts

The first two attempts at detecting the APT transmission using the FSH 4 and a log-periodic antenna failed.

Reception configuration:

- Spectrum analyzer as receiver
    - center: 137.5MHz
    - span: 1MHz
    - RBW: 3kHz
    - Att: 0 dB
- antenna: wideband log-periodic

The RBW setting might have been something else, I'm not sure. In retrospect, considering the 34 kHz width an APT transmission, the spectrum analyser setting might have not been suitable. The satellite orbits were not great either, and we were not exaclty sure in which direction south was.

**EDIT:** it is likely that the 137.5 MHz fequency was not correct, since the satellites use different downlink frequencies.

### 16:27 UTC+2 NOAA 15 orbit 91282 137.5000 MHz

This attempt seemed promising, with an orbit peak elevation of over 70 degrees, which is higher than the two earlier attemps. But, no luck.

We also tried another antenna (**TODO:** what antenna?) but saw no difference.

Reception configuration:

- FSH 4 Spectrum analyzer as receiver
    - center: 137.5 MHZ
    - span: 200 kHz
    - RBW: 3 kHz
    - Att: 0 dB
- antenna: wideband log-periodic


### 16:50 UTC+2 NOAA 18 orbit 54283 137.9125 MHz

**successfully viewed the transmission** on the spectrum analyzer. 

Reception configuration:

- FSH 4 Spectrum analyzer as receiver
    - center: 137.9125 MHZ
    - span: 100 kHz
    - RBW: 100 Hz
    - Att: 0dB
    - ref-level: -60 dBm
- antenna: wideband log-periodic
- no RF filters or amplifiers


Photo of the spctrum analyzer screen during the peak signal strength of the pass:
<img src="./photos/20151202_170044.jpg" alt="ATP reception spectur" width="400"/>

In [1]:
import math
noise_power_100Hz = 10 * math.log(138e-23 * 290 * 100, 10)  # P = kTB
noise_power_3kHz =  10 * math.log(138e-23 * 290 * 3000,10)
print(noise_power_100Hz)
print(noise_power_3kHz)

-163.977229157
-149.20601661


# 2015-12-3


## 16:40 UTC+2 NOAA18 orbit 54297 137.9125 MHz

A semi-successful test. The RTL-SDR definitely received something, but there is seems to be a signal-nosie ration problem when trying to decode the image from the baseband recording


- receiver: RTL-SRD dongle
    - demodulation: narrow FM
    - software filter: 40kHz
    - gqrx LNA set to 49 dB (maximum)
    
![screenshot of gqrx + gnome predict](./photos/test-screenshots/Screenshot from 2015-12-03 16:46:28.png)
    

# 2015-12-08

Got some coax connectors and copper board for building the band pass filter. Measured the inductance of some adjustable RF-inductors.

# 2015-12-14

Three reception attemps using log-periodic antenna and the RTL-SDR dongle. Tried saving both raw IQ data as well as narrow-FM demodulated data, although gqrx seems not to make any diffenerence between the two, and only saves narrow-FM demodulated data.

After trying to decode all of the recordings with wxtoimg, there are no successuful results.

# 2016-02-10

## Bandpass filter: iteration 1


The first iteration of the bandpass filter was built using perfboard, surface mount ceramic capacitors for the input and output tapping, and a variable capacitor and inductor for the tank circuit. The component values were calculated by hand, and verified using LTSpice.

In the scematic below C5 and L1 are adjustable components.

<img alt="filter shcematic" src="./circuits/bandpass-filter-ltspice/schematic.png" style="width: 400px;" />
![](./circuits/bandpass-filter-ltspice/response.png)

---
The simulation results were as expected, howoever in the actual circuit the amplitude response was ok frequency wise, but the passband attenuation was about -27 dB for some reason. The exact reason for this loss has not yet been determined.


**RETROSPECT**: Turns out the coax connectors used had S-connectors, which are rated for 75 $\Omega$, not 50 $\Omega$ like the both the circuit and the circuit analyzer are rated. **TODO**: Are losses this high with mismatched coax?

**RETROSPECT**: The coax leads were about 15 cm long. Having measured the loss to be less than 0.5 dB, the conclusion is that the attenuation is in the circuit, not the mismatched coax.


<img alt="photo of the filter amplitude response. It has 25 dB passband attenuation" src="./photos/20160211_133212.jpg" width="400" />

---


Below are photos of the constructed board. Surface mount components are on one side and through hole components on the other. 

<img alt="top side of the filter circuit" src="./photos/20160211_133408.jpg" style="width: 400px;" />
<img alt="top side of the filter circuit" src="./photos/20160211_133416.jpg" style="width: 400px;" />
<img src="./photos/20160211_133454.jpg" alt="Coax S-connector" width ="400"/>

# 2016-02-11

Spent some time looking into quadfilar helix antennas. Found some dimension calculators online and build instructions online:

- http://www.jcoppens.com/ant/qfh/calc.en.php
- http://homepage.ntlworld.com/phqfh1/qfh_diy_guide.htm
- http://www.housedillon.com/?p=1461


A quadfilar helix antenna would be very well suited for APT reception. The quadfilar helix antenna is **not** especially directional, but radiates fairly evenly to one hemisphere, and the polarization is circular. These factors allow recepition of a satellite with a statically mounted antenna, while maximizing the gain regardless of where in the sky the satellite is.

## Dimension from jcoppens caclulator

The size of a QFH for APT reception would be roughly a 70 cm tall cylider with a 30 cm diameter 

element                           | larger  | smaller
----------------------------------|---------| 
Total length                      | 2.40 m | 02.28 m
Vertical separator                | 0.88 m | 0.84 m
Total compensated length          | 2.41 m | 2.29 m
Antenna height                    | 0.73 m | 0.69 m
Internal diameter                 | 0.31 m | 0.30 m
Horizontal separator              | 0.32 m | 0.30 m




# 2016-03-02

Three different bandpass circuits were designed using AWR. The designs are based on the earlier interation of the bandpass filter.
The optimizer goals were set based on measurements from 2015-12-2 that were made using a wideband logperiodic antenna attached to a portable spectrum analyzer.

See the [/circuits/bandpass-filter-AWR/](./circuits/bandpass-filter-AWR/) directory the AWR CAD files.

The circuits with 2 and 3 LC oscillators were not designed very seriously, more as an exercise in using AWR and especially the optmizer feature. For example there are capacitor values smaller than smaller than 1 pF, which might not practical.


<img alt="schematic and analysis plots for bandpass filter" src="./circuits/bandpass-filter-AWR/NOAA-single-res.png" style="width: 500px;"/>



---


<img alt="schematic and analysis plots for bandpass filter" src="./circuits/bandpass-filter-AWR/NOAA-double-res.png" style="width: 500px;"/>
<img alt="schematic and analysis plots for bandpass filter" src="./circuits/bandpass-filter-AWR/NOAA-triple-res.png" style="width: 500px;"/>
	


# 2016-03-10

Wrote and refactored a bunch of documentaition and cleaned some files. 

Learned how to use `wxtoimg` from the commandline. 

```
wxtoimg -tn -N some.wav > some.wav.jpg
```

The `-tn` options sets the satellite type to NOAA. `-N` specifies a northbound pass. The passes that happen around 1600 UTC+2 are all northbound.

Decoded images from all the reception attempts can be found in the [/recrodings](./recordings) directory. As of yet, none of the recordings decode properly, they are mostly noise. Below is an example of a decoded image. The gap between the two sensor images is visible when viewing the image at native resolution.

<img alt="whitenoise image decoded from ATP transmission" src="./photos/gqrx-20151203-144531.wav.wxtoimg.jpg" style="width: 500px;" />

**RETROSPECT**: the reason for wxtoimg being unable to decode the images is almost certainly due to incompatible .wav files, not because the data is bad. The same files have been decoded with a GNU radio script.

# 2016-03-11

Some python DSP code was developed [in the DSP notebook](./signal-processing/APT-DSP.ipynb). This entry gives a high  level overview of the code's functionality.

## Image decoding algorithm

The baseband signal that is received from a satellite's FM broadcast consists of a 8-bit pulse code modulated (PCM) image data and metadata signal that has been amplitude modulated onto a 2.4 kHz subcarrier. The symbol rate of the PCM signal is  4160 symbols/second

Below is the GNU radio flow chart of the image decoder. The subsequent sections explains the reasoning behind each block.

![grc flow chart of the image decoder](./signal-processing/APT-decoder.grc.png)


### Band pass filter


A data stream of 4160 symbols/second gives a fundamental frequency up to 2080 Hz. A 2080 Hz sine wave amplitude modulated onto a 2400 Hz carrier produces frequency components at 2400 + 2080 = 4480 as well as 2400 - 2080 = 320.

The passband of the bandpass filter is set to {500...4200} Hz. This is slightly narrower than {320...4480} Hz, but it is fine, because filter's rolloff is not instantaneous.


### Subcarrier demodulation

The first step in the AM demodulator is to resample the filtered baseband signal at 4 times the subcarrier frequency: 2400 × 4 = 9600 samples/second. This gives a new representation of the signal where each period of the subcarrier is sampled at 90° phase intervals.

The momentary amplitude $A$ of a sine wave can be calculated given two samples $s_1$ and $s_2$ that have a 90° phase difference compared to eachother.  

$$A = \sqrt{s_1^2 + s_2^2} $$

The calculation is essentialy the pythagorean theorem, and is implemented in the GRC flow chart as the magnitude of a coplex vector. The imaginary and real components of the vector are the $s_1$ and $s_2$ values respectively

### Post demodulation

Demodulation of the subcarrier produces a 9600 samples/second stream of the image data, represented using floating point values in the range {0...1}. The stream is resampled at the data symbol rate: 4160 samples/second, in order to represent each data symbol with a single sample. The final samples are multiplied by 255, cast to 8-bit usigned integers, and written to a file in sequence.

Each byte in the written file prepresents a single pixel value. The file is converted to a .png using the `convert` utility from ImageMagick. A line in APT is 2080 pixels wide.

```
convert -size 2080x1500 -depth 8 test.wav.gray test.wav.png
```



## The data frequency higher than carrier frequency

As mentioned earlier, the maximum fundamental frequency of APT image data is 2080 Hz. Assuming the symbols are square pulses, there would be odd harmonics, such as 2080 Hz × 3 = 6240 Hz in the baseband.


**TODO:** The harmonics migth well have been removed before subcarrier modulation, but what would the spectrum look like if they were not?

**TODO:** How well would it modulate into a 34 kHz FM transmission


## ATP data signal waveform

Below are two graphs generated with the [DSP notebook](./signal-processing/APT-DSP.ipynb). One depicts the waveform of a single ATP line, while the other depicts a more detailed view of the middle section of the former waveform.

!["graph of APT-line-waveform-overview"](./photos/APT-line-waveform-overview.svg)
!["graph of APT-line-waveform-video-transition"](./photos/APT-line-waveform-video-transition.svg)

The purpose of the waveform generator script in the DSP notbook was to create a way of getting perfect APT transmissions that can be used to test who well different image decoding algorithms work. The scripts generates an APT datastream in memory, and modulates it onto the 2400 Hz AM subcarrier, based on the APT specification given in Section 4.2 of the NOAA KLM User's Guide. The waveform is saved in .wav format, so that it can be used by other applications.

One of the video channels in the test images contains only noise. 

## Decoding images


The images below were produced by supplying a recoded .wav to a custom GNU radio python script.


Pristine image decoded from a perfect baseband signal generated with the DSP notebook:
!["The first test image decoded using GNU radio"](./photos/first-decoded-test-image.png)

Below is an image of unaltered output from the GNU radio script using an actual recording from a satellite pass. The recording was not great, but there are clearly visible clouds in the video data. The telemetry and frame sync data is also clearly identifiable. The horizontal shear in the image is likely due to the simplicity of the decoding algorithm. It would **propably** be fixed  by using carrier recovery for the subcarrier, instead of a hardcoded 2400 Hz frequency.
!["first sucessfully decoded image from satellite. Using GNU radio"](./photos/first-decoded-satellite-image.png)


**TODO**: figure out what the reason for the shear really is.


# 2016-03-12

Tried to receive NOAA 19 (orbit 36562) with a simple $\lambda / 4 $monopole. I did not have access to a suitable location for reception, so I tried receiveing the transmission from a parking lot between buildings. The satellite signal did not appear in the spectrum view in gqrx. NOAA19 might have been moved to 137.6 MHz from 137.5 MHz, where I was listening.

**RETROSPECT**: The monople did not have a proper ground plane either. Very little went right on this test.

----

Some more python code was written. See [/orbit-data/orbit-plot.ipynb](./orbit-data/orbit-plot.ipynb) for a notebook that contains some plots of the elevation/azimuth data for the orbits that have been used to attempt to receive images from satellites.


# 2016-03-16

Built some simple antennas, and compared how well they worked.

Some input matching measurements of the RTL-SDR dongle were also done, using a circuit analyzer.


## RTL-SDR input matching


This section documents the input matching measurements of the RTL-SDR in a 50 $\Omega$ system using a **(TODO: what)** circuit analyzer. 


The S11 value was measured to be in the range {-11...-13} dB in the {137...138} MHz band. The S11 value was constant along the band, but seemed to vary depeding on how well the coax connector coupled to the RTL-SDR. Measurements were done with a -20 dBm sweep power, as this is the sweep power used in the R802T chip datasheet.

The measurements coincide with the spcifications of the R802T integrated circuit tuner, which is the first stage in the RTL-SDR. The datasheet specifies S11 < -10 dB on all operating frequencies.

The value S11 = -12 dB gives a plain reflection  $\rho  = 10^{-12/10} = 0.0631$. This further gives a standing wave ratio of:

$SWR = \cfrac{1 + \rho}{1 - \rho} \approx 1.13$ 

**TODO**: clear up differences between VSWR vs SWR. 


---

Occasionally the S11 values would stay at a constant S11 = -0.5 dB. The attenuation of the coax used to connect the receiver to the analyzer was S21 = -0.3 dB. This would make the S11 = -0.5 dB measured for the RLT-SDR indicative of the coax connector on the receiver not being coupled properly (open circuit). The reciever was still able to do the AD conversion, though, and show the circuit analyzer's signal sweeping across the 1MHz span at 137.5 MHz.

**RETROSPECT**: while the input matching measurements are not wrong, they should be done using the smith chart functionality on the analyzer, in order to get all the data, not just the vector magnitude

**TODO**: redo measurements using smith chart


---




## Antenna comparison tests

Below are some comparison measurements that were were done for the original log-periodic antenna, a custom half-wave dipole , and a custom quarter-wave monopole. The dipole and monopole antennas were constructed using 1.5 mm insulated copper wire. 

The antennas were rotated around the vertical axis in order to find a direction where maximum signal strength could be observed. Measurements were done with the atennas aligned both horizontally and verically. The LNA gain in gqrx was set to +17 dB.

5 signal strength values were observed for each test. The antenna was moved around slightly between observations, while still keeping it aligne in the direction.

Values are in dBFS (full scale) as given by the gqrx receirver program.


antenna |min | max | mean
--------|----|-----|-----
horzontal dipole | -57 | -52 | -55.0
vertical dipole | -40 | -35 | -37.5
horizonta monopole | -50 | -42 | -45.5
vertical monopole | -43 | -37 | -40.25
horizontal logperiodic | -54 | -50 | -51.75
vertival logperiodic | -52 | -51 | -51.25

The AITS beacon from Turku airport was used as test signal. The ATIS beacon was chosen as the test signal because it is relatively strong, and close to the 137 MHz frequency of APT transmissions. The ATIS beacon is on 130 MHz. The ATIS is transmission is not circularly polarized, which is a key difference when comparing to APT transmissions.

**RETORSPECT**: The measurements were made from a reflection, not directly from the ATIS beacon. The antenna headings (where applicable) were approximately -45 degrees offset from the ATIS beacon heading. The test results should sitll be viable, though, since the same heading was used for all the tests.  


![Panorama of the test site](./photos/20160316_153528.jpg)
![Panorama of the test site](./photos/20160316_153548.jpg)

# 2016-03-22

Took a closer look at the RTL-SDR dongle. Turns out the first stage the RF signal is fed to is an R820T, not the RTL2932U. RTL-SDR measurements in [notes.ipynb](./notes.ipynb) will be compared the what is in the datasheet.

Reading through the datasheet gave some insights as to who some of the charasteristics of the tuner/mixer circuits are are measured modelled. E.g I had not seen phase noise given as dBc/Hz @ some frequency before, altough in hindsigth it seems like a very reasonable unit.  


# 2016-03-23

Read about geosataionary orbits and looked for other systems similar to the NOAA POES constellation. Unfortunately NOAA GOES geostationary satellites are sationary over the Americas, and therefore not available for reception in Finland.

The EUMETSAT program has interesing wheather and science satellites in goestationary orbits above Africa, and in polar orbits.



# 2016-03-25

Excursion to Kornoinen for antenna and reciver configuration field tests. One test completed.


- orbit data: [/obrit-data/NOAA-19-36731.txt](./orbit-data/NOAA-19-36731.txt)
- antenna: DIY half-wave dipole suspended horizontally half a meter above (a quarter wavelength) ground. The elements were aligned perpendicularly to the satellite's ground track.
- receiver: RTL-SDR dongle.
- receiver settings: Using gqrx for demodulation. Gqrx AGC disabled. Grqx LNA gain setting manually adjusted at the range {30...35} dB during majority of pass. LNA in the {40...49} dB range just before loss of signal. Software bandwidth filter set at 39.6 kHz. Manual doppler frequency shift compenstation.
- wheather: cloudy, minimal rain
- system: antenna -> 5 meters of 50 ohm coax -> RTL-SDR



![panorama photo of the test site](./photos/20160325_135407.jpg)
![panorama photo of the test site](./photos/20160325_135421.jpg)



## Decoded image

<img alt="georeferenced satellite image" src="./photos/gqrx-20160325-113620.wav-enhanced.png"  style="width: 500px;"/>

# 2016-03-26

Excursion to Vartiovuori for antenna and receiver configuration field tests. The test setup was approximately the same as the previous day. One (and a half) out of three planned test were completed. 

## Test equipment

- antenna: DIY half-wave dipole
- receiver: RTL-SDR dongle
- wheather: slight overcast
- system: antenna -> 5 meters of 50 ohm coax -> RTL-SDR

## Test 1

- orbit data: [/orbit-data/NOAA-18-55906.txt](./orbit-data/NOAA-18-55906.txt)

**receiver settings**: Using gqrx for demodulation. Gqrx AGC disabled. Grqx LNA gain setting manually adjusted in the range {30...35} dB during majority of pass. LNA in the {40...49} dB range just before loss of signal. Software bandwidth filter set at 39.6 kHz. Manual doppler frequency shift compenstation.

The elements of the diploe antenna were suspended horizontally approximately one quarter wavelength above the ground. No mast was available.


The diplole was aligned perpendicular to the direction of the satellite's ground path.

<img alt="decoded image 2016-03-26 test 1" src="./recordings/gqrx-20160326-142943.wav.png" style="width: 500px;" />


## Test 2

- orbit data: [/orbit-data/NOAA-19-36747.txt](./orbit-data/NOAA-19-36747.txt)

The purpose of the second test was to test how well images could be received from non-optimal satellite passes. The maximum elevation of the pass was 7,76 degrees. The BNC coax adapter for the RTL-SDR broke from physical strain during the second satellite pass. The test had to be interrupted because of this.

Regardeless of the equipment failure, some data was sitll captured.

<img alt="decoded image 2016-03-26 test 2" src="./recordings/gqrx-20160326-144940.wav.png" style="width: 500px;" />



---




![panorama photo of the test site](./photos/20160326_165852.jpg)
![panorama photo of the test site](./photos/20160326_165910.jpg)


# 2016-03-31

Two iterations of the bandpass filter were built, based on the single LC oscillator design from [2016-03-02](#2016-03-02). The wound inductor in the first circuit was not quite right, and the filter's center frequency could not be tuned above approximately 122 MHz.

A new inductor was wound with one turn less. The inductors were wound from 1.2 mm diameter enamel coated copper wire around a 4 mm diameter cylinder. The first inductor consisted of 6 turns, and the latter of 5 turns

Due to the lack of suitable coax adapters, the circuit was connected to the analyzer roughly 2 meters of coax, resulting in 2.1 dB attenuation in the passband. The analyzer was not calibrated to take this into account. The attenuation is pesent in the measurement data below.

Compensating for the loss in the coax cabel, the filter's insertion loss is 6.6 dB - 2.1 dB = 4.5 dB



---

<img alt="" src="./photos/20160331_174023.jpg" style="width: 400px;" />
<img alt="" src="./photos/20160331_174246.jpg" style="width: 400px;" />
<img alt="" src="./photos/20160331_174746.jpg" style="width: 400px;" />
<img alt="" src="./photos/20160331_175141.jpg" style="width: 400px;" />


# 2016-04-01

Another iteration of the bandpass filter circuit was constructed. Perfomance was slightly better than the [2016-03-31](#2016-03-31) version (insertion loss 3 dB, reflection -11dB). This version was buit using a different, physically larger inductor and variable capacitor. The center frequency of the filter was around 98 MHz at its best. Another version will be built using less turns in the inductor.



<img alt="photo of filter circuit" src="./photos/20160401_111907.jpg" style="width: 400px;" />

**TODO**: change the inductor to one with less turns.  

# 2016-04-02

This entry documents analyses of some of the characteristics of the APT baseband signal. Measurements and analysis was done using [baudline](http://www.baudline.com), a general purpose siganl analysis tool.

The image below represents the spectrum of an APT baseband signal. The spectrum is the average of 2 minutes of a recorded satellite pass, specifically the pass recorded during test 1 on [2016-03-26](#2016-03-26). The table lists the frequencies of the most significant peaks in the spectrum.


<img alt="APT baseband spectrum average" src="./photos/baudline-ATP-baseband-average-spectrum.png" />

The APT specification specifies two sync signals, one consisting of 7 consecutive 3/5 duty ratio pulses at 832 Hz, and another consisting of 7 consecutive 1/2 duty ration pulses at 1040 Hz. 

frequency | origin | reasoning
----------|--------
96    | 2400 - 832 × 3 = -96 | videoB sync third fundamental
736   | 2400 - 832 × 2 | videoB sync second harmonic 
1360  | 2400 - 1040 | videoA sync fundamental
1568  | 2400 - 832  | videoB sync fundamental
2400  | 2400 | subcarrier
3232  | 2400 + 382 | videoB sync fundamental
3440  | 2400 + 1040 | videoA sync signal fundamental
4064  | 2400 + 382 × 2 | videoB second harmonic
4800  | 2400 × 2 | subcarrier second harmonic
4896  | 2400 + 832 × 3 | videoB sync third harmonic
5520  | 2400 + 1040 × 3 | videoA sync third harmonic
5728  | 2400 + 832 × 4 | videoB fourth harmonic
7200  | 2400 × 3 | subcarrier third harmonic
9600  | 2400 × 4 | subcarrier fourth harmonic
12000 | 2400 × 5 | subcarrier fifth harmonic
16800 | 2400 × 6 | subcarrier sixth harmonic
21600 | 2400 × 7 | subcarrier seventh harmonic


----

There is clear drift in the subcarrier frequency in the recording. The subcarrier frequency was measured using as a function of time using baudline. Datapoints were measured at approximately equal intervals, with additional datapoints added where there were sudden changes.

Gqrx, the software that was used to record the analysed sample, does not include a feature that allows it to track the APT transmission's carrier frequency (RF carrier) when it drifts due to doppler shift. Compensation for the drift was done manually during the recording, which is likely the reason the why the datapoints are erratic after the 10 minute mark. 

**TODO**: record a pass while ignoring doppler shift entirely, in order to get a clean sample without manual compensation.



![APT subcarrier drift as a function of time](./photos/subcarrier-drift-as-function-of-time.png)


The working theory is that the drift in the subcarrier frequency is what causes the images produced by the GNU radio flow chart to have horizontal shear.

# Time overview

In [4]:
import datetime
time = [
    [16.00, 20.00, datetime.date(2015,11,30)], 
    [16.00, 20.00, datetime.date(2015,11,29)],
    [16.00, 20.00, datetime.date(2015,11,28)], # thes fors ones are just some guesses
    [16.00, 20.00, datetime.date(2015,12,1 )], # although these later ones are not the accurate either
    [09.00, 17.50, datetime.date(2015,12,2 )], # Whoo! first contact
    [19.00, 20.00, datetime.date(2015,12,2 )], # writing documentation about first contact
    [08.00, 11.00, datetime.date(2015,12,3 )], # designing rf bandpass filter
    [11.30, 15.00, datetime.date(2015,12,3 )], # calculating coil sizes and stuff
    [15.00, 17.00, datetime.date(2015,12,3 )], # prepping for test no. 5
    [12.00, 13.40, datetime.date(2015,12,3 )], # bandpass filter build techniqe planning 
    [15.50, 18.50, datetime.date(2015,12,14)], # tests 6,7,8, and trying to decode recordings
    [14.50, 16.00, datetime.date(2016, 2,8 )], # Simulating bandpassfilter using AWR 
    [09.00, 12.00, datetime.date(2016, 2,10)], # built bandpass filter prototype, dealt with caax connectors
    [12.50, 13.70, datetime.date(2016, 2,11)], # competed first bandpass protopyte, analyzed it. bad results
    [15.50, 16.50, datetime.date(2016, 2,11)], # looking into antennas, reading existing documentation
    [18.30, 19.75, datetime.date(2016, 2,11)], # looking into antennas, palning on building QFH
    [ 9.00, 12.15, datetime.date(2016, 3,2 )], # after the bad results from the earlier prototype, made new simulations using AVR
    [14.00, 19.00, datetime.date(2016, 3,10)], # improved documentation and organized files. learned ot use wxtoim commandline
    [0.0,   6.00 , datetime.date(2016, 3,11)], # python DSP code for generating APT baseband signals
    [7.0,   9.0  , datetime.date(2016, 3,11)], # antenna stuff + link budget
    [13.0,  16.00, datetime.date(2016, 3,11)], # DSP stuff for generating APT data signals
    [16.00, 17.5 , datetime.date(2016, 3,11)], # generate bitmaps from APT data signals
    [18.25, 22.0 , datetime.date(2016, 3,11)], # DSP can now generate WAV files. wxtoim almost decodes them
    [22.30, 23.50, datetime.date(2016, 3,11)], # Designed a simple APT decoder in gnuradio. It sucessfully decoded the first image
    [08.50, 10.00, datetime.date(2016, 3,12)], # try to use the RTL-SDR block in DRC
    [10.00, 11.33, datetime.date(2016, 3,12)], # tried a few monopoles on 130.05 MHz, plannin field test 
    [14.00, 14.30, datetime.date(2016, 3,12)], # failed to receive form NOAA 19 using λ/4 monopole
    [15.50, 16.00, datetime.date(2016, 3,12)], # decoded all satellite recordings into images
    [19.00, 21.00, datetime.date(2016, 3,12)], # Draw orbit data plots
    [09.00, 11.00, datetime.date(2016, 3,16)], # Reading about constructing HIGH-Q inductors
    [11.00, 13.50, datetime.date(2016, 3,16)], # Antenna construction (di- and monople) with bnc connectos
    [14.00, 16.50, datetime.date(2016, 3,16)], # rtl input S11 reflection measurements
    [16.50, 17.25, datetime.date(2016, 3,16)], # reception of NOAA 18 using dipole antenna
    [18.00, 18.50, datetime.date(2016,3,16)], # Refining notes and documentation from the day
    #[# # ],
    [09.00, 13.00, datetime.date(2016,3,20)], # documenting signal processing, look into post processing
    [16.00, 17.25, datetime.date(2016,3,22)], # searching/reading deepinfo/datasheets for RTL-SDR
    [10.00, 12.25, datetime.date(2016,3,25)], # looking into GOES, METEOSAT and other systems
    [13.00, 13.80, datetime.date(2016,3,25)], # one field test in Kornoinen
    [16.10, 17.00, datetime.date(2016,3,26)], # two field tests on Vartiovuori
    [15.50, 18.00, datetime.date(2016,3,30)], # Built and measured an interation of the AWR optimizes single oscillator filter
    [18.50, 22.75, datetime.date(2016,3,30)], # documented field tests for 2016-03-25 and 26
    [09.75, 12.00, datetime.date(2016,4, 1)], # Built another iteration of the the single oscillators filter, looked for aplifier components
    [14.00, 17.50, datetime.date(2016,4, 1)], # documenting field tests ant bandpass filter implementations
    [10.00, 12.00, datetime.date(2016,4, 2)], # amending documentation
    [18.00, 21.50, datetime.date(2016,4, 2)], # baseband drift analysis using baudline
]

len = [row[1] - row[0] for row in time]
print "total time spent:", sum(len), "[hours]"


total time spent: 113.43 [hours]
